In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("../data/MSLR/MSLR-WEB10K/Fold1/vali.txt", header=None, sep=" ")
df_samp = train.sample(4)
df_samp.head(2)

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
51387,1,qid:7165,1:4,2:1,3:2,4:2,5:4,6:1,7:0.25000,8:0.50000,...,128:68,129:12,130:2741,131:57029,132:6,133:5,134:0,135:0,136:0,NaN
15500,0,qid:2170,1:2,2:0,3:0,4:0,5:2,6:1,7:0,8:0,...,128:80,129:0,130:153,131:376,132:8,133:10,134:0,135:0,136:0,NaN


In [3]:
def df_transform(df):
    df[1] = df[1].apply(lambda x: x[4:])
    df[df.columns[2:-1]] = df[df.columns[2:-1]].applymap(lambda x: x.split(':')[1])
    df = df.drop(138, axis=1)
    return df

In [4]:
train_df = df_transform(train)

In [29]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,0,10,2,0,0,0,2,0.666667,0,0,...,45,1,0,117,55115,7,2,0,0,0
1,0,10,1,0,1,3,3,0.333333,0,0.333333,...,76,0,0,153,3866,17,104,0,0,0
2,1,10,3,0,3,0,3,1,0,1,...,73,0,9,266,56137,5,2,0,0,0
3,0,10,3,0,2,0,3,1,0,0.666667,...,54,8,0,541,12621,11,11,0,0,0
4,1,10,3,0,3,0,3,1,0,1,...,36,6,0,14687,40205,5,3,0,0,0


In [6]:
train_df = train_df.astype(float)
train_df[1] = train_df[1].astype(int)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,0.0,10,2.0,0.0,0.0,0.0,2.0,0.666667,0.0,0.000000,...,45.0,1.0,0.0,117.0,55115.0,7.0,2.0,0.0,0.0,0.0
1,0.0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,76.0,0.0,0.0,153.0,3866.0,17.0,104.0,0.0,0.0,0.0
2,1.0,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,73.0,0.0,9.0,266.0,56137.0,5.0,2.0,0.0,0.0,0.0
3,0.0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,54.0,8.0,0.0,541.0,12621.0,11.0,11.0,0.0,0.0,0.0
4,1.0,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,36.0,6.0,0.0,14687.0,40205.0,5.0,3.0,0.0,0.0,0.0


In [31]:
x = train_df[1].value_counts()
print(x[(x < 100) & (x > 4)].count)

<bound method Series.count of 1
19000    99
7450     99
8770     99
26320    99
3400     99
         ..
28345     6
22480     5
19315     5
19525     5
15235     5
Name: count, Length: 865, dtype: int64>


In [37]:
bins = [i for i in range(0, 260, 10)]
labels = [i for i in range(0, 25)]
train_df['binned'] = pd.cut(train_df[133], bins=bins, labels=labels)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9928\1591625706.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['binned'] = pd.cut(train_df[133], bins=bins, labels=labels)


In [38]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,binned
0,0.0,10.0,2.0,0.0,0.0,0.0,2.0,0.666667,0.0,0.000000,...,1.0,0.0,117.0,55115.0,7.0,2.0,0.0,0.0,0.0,0
1,0.0,10.0,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.0,153.0,3866.0,17.0,104.0,0.0,0.0,0.0,1
2,1.0,10.0,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,9.0,266.0,56137.0,5.0,2.0,0.0,0.0,0.0,0
3,0.0,10.0,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,8.0,0.0,541.0,12621.0,11.0,11.0,0.0,0.0,0.0,1
4,1.0,10.0,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,6.0,0.0,14687.0,40205.0,5.0,3.0,0.0,0.0,0.0,0
